In [1]:
from shapely.geometry import Point, LineString, Polygon

In [ ]:
def get_exterior_boundary_from_polygon(polygon: Polygon):
    #this function returns all edges of a polygon as shapely LineString in a list
    coords = polygon.exterior.coords
    total = len(coords)
    lines = []
    for i in range(total - 1):
        lines.append(LineString(coords[i:i+2]))
    return lines

In [ ]:
def scan_polygon_to_grid(polygon: Polygon, interval: float):
    #convert polygon into a bit map.
    #the resolution is interval
    #a pixel represents a square polygon whose side length is equal to interval
    #a pixel is 1 when it is intersected with the polygon or any vertex of the polygon is inside the pixel, otherwise it is 0
    bounds = polygon.bounds
    x0, y0, x1, y1 = bounds[0], bounds[1], bounds[2], bounds[3]
    grid = []
    x_curr = x0
    y_curr = y0
    x_next = x_curr + interval
    y_next = y_curr + interval
    while y_next < y1:
        row = []
        while x_next < x1:
            pixel_curr = Polygon([
                (x_curr, y_curr),
                (x_curr, y_next),
                (x_next, y_next),
                (x_next, y_curr)
            ])
            if is_intersect_or_contain(pixel_curr, polygon):
                row.append(1)
            else:
                row.append(0)
            x_curr = x_next
            x_next += interval
        grid.append(row)
        y_curr = y_next
        y_next+= interval
        x_curr = x0
        x_next = x_curr+interval
    return grid

In [2]:
def is_intersect_or_contain(p_src: Polygon, p_tar: Polygon):
    #check if p_src intersects with p_tar or any point of p_tar is inside p_src
    for point in p_tar.exterior.coords:
        if p_src.contains(Point(point)):
            return True
    for edge in get_exterior_boundary_from_polygon(p_tar):
        if edge.intersects(p_src):
            return True
    return False